## Installing dependencies. Useful for running on Google Colab

In [1]:
# pyarrow might be needed to read the data
!python -m pip install Cython
!python -m pip install -e git+https://git@github.com/alercebroker/turbo-fats#egg=turbofats
!python -m pip install -e git+https://git@github.com/alercebroker/mhps#egg=mhps
!python -m pip install -e git+https://git@github.com/alercebroker/P4J#egg=P4J
!python -m pip install pyarrow
!python -m pip install -e git+https://git@github.com/alercebroker/lc_classifier#egg=lc_classifier

Obtaining turbofats from git+https://****@github.com/alercebroker/turbo-fats#egg=turbofats
  Cloning https://****@github.com/alercebroker/turbo-fats to ./src/turbofats
  Running command git clone -q 'https://****@github.com/alercebroker/turbo-fats' /content/src/turbofats
  Running setup.py develop for turbofats
Obtaining mhps from git+https://****@github.com/alercebroker/mhps#egg=mhps
  Cloning https://****@github.com/alercebroker/mhps to ./src/mhps
  Running command git clone -q 'https://****@github.com/alercebroker/mhps' /content/src/mhps
  Running setup.py develop for mhps
Obtaining P4J from git+https://****@github.com/alercebroker/P4J#egg=P4J
  Cloning https://****@github.com/alercebroker/P4J to ./src/p4j
  Running command git clone -q 'https://****@github.com/alercebroker/P4J' /content/src/p4j
  Running setup.py develop for P4J
Obtaining lc_classifier from git+https://****@github.com/alercebroker/lc_classifier#egg=lc_classifier
  Cloning https://****@github.com/alercebroker/lc_cla

## [Google Colab] Please restart Python after installing the packages:
### Runtime >> Restart runtime

In [1]:
import numpy as np
import pandas as pd
from lc_classifier.features import ZTFLightcurvePreprocessor, ZTFFeatureExtractor

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Reading the sample data

To calculate the model features we will need:
- Detections: Alerts from the ZTF stream, with 5 $\sigma$ difference between the template and the current image.
- Non Detections: Previous measurements of the object but with less than 5 $\sigma$ magnitudes.
- Object Information: Aggregated information by ALeRCE.

In [4]:
detections = pd.read_parquet(
    'https://github.com/alercebroker/lc_classifier/raw/main/examples/'
    'data/detections_sample.parquet')
non_detections = pd.read_parquet(
    'https://github.com/alercebroker/lc_classifier/raw/main/examples/'
    'data/non_detections_sample.parquet')
object_information = pd.read_parquet(
    'https://github.com/alercebroker/lc_classifier/raw/main/examples/'
    'data/object_info_sample.parquet')

In [5]:
detections.head()

,fid,isdiffpos,sigmapsf,field,fwhm,dec,magpsf,rcid,ra,sky,rb,ssmagnr,distpsnr3,sgscore2,maggaiabright,distpsnr2,distpsnr1,maggaia,exptime,drb,sgscore3,neargaia,sgscore1,mjd,corrected,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext,dubious,has_stamp
oid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,1,-1.0,0.098972,396,2.910000,0.802001,17.957479,61,358.760453,-0.130846,0.910000,-999.0,18.236210,0.026667,-999.0,12.874929,0.089459,15.460238,30.0,0.989089,0.441208,0.061395,0.99875,58700.461042,True,16.478576,0.008541,0.025349,False,True
ZTF17aaaemke,1,-1.0,0.075092,396,3.850000,0.801930,17.806704,61,358.760434,0.125953,0.891429,-999.0,18.109360,0.026667,-999.0,12.938040,0.304864,15.460238,30.0,0.999953,0.441208,0.274704,0.99875,58754.290857,True,16.520813,100.000000,0.022974,False,True
ZTF17aaaemke,1,-1.0,0.093225,396,3.160000,0.801997,17.616571,61,358.760472,0.457313,0.940000,-999.0,18.180468,0.026667,-999.0,12.938669,0.031614,15.460238,30.0,0.999997,0.441208,0.005324,0.99875,58763.377998,True,16.586325,0.024660,0.036094,False,True
ZTF17aaaemke,1,-1.0,0.146396,396,1.544062,0.801997,18.686962,61,358.760405,-0.294222,0.891429,-999.0,18.350967,0.026667,-999.0,12.729602,0.262691,15.460238,30.0,0.999442,0.441208,0.233289,0.99875,58718.413889,True,16.350400,100.000000,0.017018,False,True
ZTF17aaaemke,1,-1.0,0.091417,396,3.670000,0.801892,17.775181,61,358.760452,0.110218,0.616667,-999.0,17.966200,0.026667,-999.0,13.063952,0.409953,15.460238,0.0,0.000000,0.441208,0.386295,0.99875,58372.401910,True,16.530643,0.014738,0.029054,False,True


In [6]:
non_detections.head()

,parent_candid,jd,fid,pid,diffmaglim,pdiffimfilename,programpi,programid,candid,isdiffpos,tblid,nid,rcid,field,xpos,ypos,ra,dec,magpsf,sigmapsf,chipsf,magap,sigmagap,distnr,magnr,sigmagnr,chinr,sharpnr,sky,magdiff,fwhm,classtar,mindtoedge,magfromlim,seeratio,aimage,bimage,aimagerat,bimagerat,elong,nneg,nbad,rb,ssdistnr,ssmagnr,ssnamenr,sumrat,magapbig,sigmagapbig,ranr,decnr,scorr,magzpsci,magzpsciunc,magzpscirms,clrcoeff,clrcounc,rbversion,mjd
oid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,618401906115010002,2.458353e+06,1,598377030615,20.065100,/ztf/archive/sci/2018/0822/376991/ztf_20180822...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58352.377037
ZTF17aaaemke,636353206115010001,2.458361e+06,1,606403590615,19.109699,/ztf/archive/sci/2018/0830/403600/ztf_20180830...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58360.403600
ZTF17aaaemke,618401906115010002,2.458364e+06,1,609312890615,20.057800,/ztf/archive/sci/2018/0902/312859/ztf_20180902...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58363.312894
ZTF17aaaemke,618401906115010002,2.458367e+06,1,612400500615,20.791800,/ztf/archive/sci/2018/0905/400509/ztf_20180905...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58366.400509
ZTF17aaaemke,618401906115010002,2.458370e+06,1,615325740615,20.812700,/ztf/archive/sci/2018/0908/325729/ztf_20180908...,Kulkarni,1,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,t12_f5_c3,58369.325741


In [7]:
object_information.head()

,ndethist,ncovhist,mjdstarthist,mjdendhist,meanra,meandec,sigmara,sigmadec,firstmjd,lastmjd,deltamjd,step_id_corr,nearZTF,nearPS1,stellar,corrected,ndet,ndubious,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr
oid,,,,,,,,,,,,,,,,,,,,,,
ZTF17aaaemke,135.0,670.0,58302.481944,59107.311562,358.760440,0.801990,0.000086,0.000060,58372.401910,59107.311562,734.909653,corr_bulk_0.0.1,True,False,True,True,88,0,1.209530,0.828800,0.531433,0.952658
ZTF17aaafyya,968.0,1471.0,58101.201285,59107.366215,17.787271,61.129036,0.000128,0.000066,58314.467755,59107.366215,792.898461,corr_bulk_0.0.1,True,True,True,True,375,3,0.757556,0.824260,0.635323,0.916122
ZTF17aaageae,280.0,783.0,58351.390069,59107.310625,8.686962,0.905769,0.000107,0.000067,58355.371991,59107.310625,751.938634,corr_bulk_0.0.1,True,True,True,True,217,0,0.168999,-0.030895,0.063114,0.207873
ZTF17aaaivsr,232.0,669.0,58101.410752,58981.216250,129.397864,9.654056,0.000060,0.000064,58375.516100,58981.216250,605.700151,corr_bulk_0.0.1,True,True,True,True,79,0,0.663141,0.630097,0.823536,0.559042
ZTF17aaaizej,332.0,428.0,58101.419444,58987.201377,141.224907,4.263769,0.000071,0.000050,58472.420324,58987.201377,514.781053,corr_bulk_0.0.1,True,True,True,True,100,0,-0.603085,-0.105530,0.113668,0.197381


# Calculating features

## Light curve preprocessing

In [8]:
preprocessor = ZTFLightcurvePreprocessor(stream=False)

detections = preprocessor.preprocess(detections, objects=object_information)
non_detections = preprocessor.rename_columns_non_detections(non_detections)

## Feature computation

In [9]:
feature_extractor = ZTFFeatureExtractor(bands=(1, 2), stream=False)

In [10]:
# We use the compute_features method
features = feature_extractor.compute_features(
    detections = detections,
    non_detections = non_detections)

In [11]:
# The result is a dataframe with the features
features

,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,Con_2,Eta_e_1,Eta_e_2,ExcessVar_1,ExcessVar_2,GP_DRW_sigma_1,GP_DRW_sigma_2,GP_DRW_tau_1,GP_DRW_tau_2,Gskew_1,Gskew_2,Harmonics_mag_1_1,Harmonics_mag_1_2,Harmonics_mag_2_1,Harmonics_mag_2_2,Harmonics_mag_3_1,Harmonics_mag_3_2,Harmonics_mag_4_1,Harmonics_mag_4_2,Harmonics_mag_5_1,Harmonics_mag_5_2,Harmonics_mag_6_1,Harmonics_mag_6_2,Harmonics_mag_7_1,Harmonics_mag_7_2,Harmonics_mse_1,Harmonics_mse_2,Harmonics_phase_2_1,Harmonics_phase_2_2,Harmonics_phase_3_1,Harmonics_phase_3_2,...,g-r_max,g-r_max_corr,g-r_mean,g-r_mean_corr,gal_b,gal_l,iqr_1,iqr_2,last_diffmaglim_before_fid_1,last_diffmaglim_before_fid_2,last_mjd_before_fid_1,last_mjd_before_fid_2,max_diffmaglim_after_fid_1,max_diffmaglim_after_fid_2,max_diffmaglim_before_fid_1,max_diffmaglim_before_fid_2,mean_mag_1,mean_mag_2,median_diffmaglim_after_fid_1,median_diffmaglim_after_fid_2,median_diffmaglim_before_fid_1,median_diffmaglim_before_fid_2,min_mag_1,min_mag_2,n_det_1,n_det_2,n_neg_1,n_neg_2,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,n_pos_1,n_pos_2,positive_fraction_1,positive_fraction_2,r-W2,r-W3,rb,sgscore1
ZTF17aaaemke,0.319945,0.277661,0.999998,0.999999,1.0,1.0,0.450000,0.382353,0.026316,0.0,0.067383,0.972511,0.000098,0.000151,0.027930,0.036893,0.021441,2.286152,-0.212015,-0.038029,0.296083,0.225232,0.432700,0.079171,0.174984,0.042829,0.269495,0.095460,0.204885,0.038857,0.065695,0.051123,0.109944,0.026285,2.226291e-03,1.697028e-03,0.481856,3.138625,3.462746,1.243828,...,0.673079,0.828800,0.511538,0.977816,-58.949304,94.752756,0.145523,0.369271,20.754400,19.582899,58372.287130,58431.186701,20.858801,20.700300,20.812700,20.459200,16.491376,15.513559,20.200001,19.968800,20.592699,20.179701,16.080139,15.251339,40.0,34.0,35.0,22.0,105.0,103.0,7.0,7.0,5.0,12.0,0.125000,0.352941,2.18798,2.92098,0.875000,0.998750
ZTF17aaafyya,0.376849,0.344548,1.000000,1.000000,1.0,1.0,0.410072,0.421569,0.000000,0.0,0.355736,0.001499,0.000308,0.000318,0.074034,0.067761,0.039956,0.081235,-0.132926,-0.018580,0.310819,0.292621,0.082010,0.095216,0.006734,0.018867,0.008706,0.017764,0.020393,0.012451,0.020567,0.024264,0.010451,0.014306,4.303091e-03,2.537017e-03,6.201483,0.015089,4.344247,5.012808,...,0.757556,0.851637,0.773540,0.876551,-1.651650,125.309979,0.538666,0.524727,NaN,20.834400,NaN,58368.344109,21.011299,20.883200,NaN,20.834400,15.544264,14.667713,19.968601,20.096550,NaN,20.834400,15.193118,14.341481,139.0,102.0,79.0,54.0,90.0,82.0,0.0,1.0,60.0,48.0,0.431655,0.470588,NaN,NaN,0.790000,0.996250
ZTF17aaageae,0.219709,0.338611,1.000000,1.000000,1.0,1.0,0.333333,0.105263,0.000000,0.0,0.020933,0.021095,0.000150,0.000160,0.030440,0.032353,0.476989,0.287248,0.098121,0.146700,0.259460,0.777605,0.074579,0.728683,0.153137,0.560363,0.055851,0.436916,0.122012,0.335144,0.029095,0.203292,0.049782,0.074806,6.606257e-04,8.862462e-03,2.236083,5.903678,4.048231,5.782700,...,0.168999,-0.030895,-0.043280,0.176898,-61.679799,114.110787,0.345228,0.265678,20.325500,20.385900,58364.415741,58352.460579,20.864300,20.714899,20.325500,20.385900,14.545857,14.368959,20.154699,19.997299,20.124401,20.385900,14.122148,14.153042,51.0,38.0,22.0,22.0,136.0,177.0,2.0,1.0,29.0,16.0,0.568627,0.421053,1.26633,2.41333,0.748571,1.000000
ZTF17aaaivsr,0.343258,0.305270,0.999999,0.821926,1.0,1.0,0.371429,0.400000,0.000000,0.0,1.747418,4.297083,0.000175,0.000131,0.049481,0.034887,0.107830,0.003242,-0.118029,-0.113409,0.280162,9.717215,0.191826,11.452487,0.183256,11.207057,0.126153,8.726390,0.146861,6.441762,0.063273,3.569088,0.069324,0.984636,2.894865e-03,2.153488e-03,5.377133,0.239235,3.205709,0.673628,...,0.663141,0.615749,0.732578,0.575847,27.935734,216.280042,0.435103,0.231598,20.449301,NaN,58465.529387,NaN,20.624201,20.575300,20.449301,NaN,16.853123,16.277276,19.621601,19.868601,20.085449,NaN,16.535991,15.920242,35.0,25.0,22.0,21.0,24.0,71.0,2.0,0.0,13.0,4.